In [1]:
import os

# Set the OpenAI API key globally
os.environ["OPENAI_API_KEY"] = "YOURKEY"
# Set Google API key and Search Engine ID directly in the notebook
os.environ["GOOGLE_API_KEY"] = "YOURKEYI"
os.environ["GOOGLE_SEARCH_ENGINE_ID"] = "YOURUD"


In [2]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:
    import os
    import time
    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")
    
    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": api_key, "cx": search_engine_id, "q": query, "num": num_results}

    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            return f"Error fetching {url}: {str(e)}"

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append({"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body})
        time.sleep(1)

    return enriched_results


In [3]:
def arxiv_search(query: str, max_results: int = 2) -> list:
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append({
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "published": paper.published.strftime("%Y-%m-%d"),
            "abstract": paper.summary,
            "pdf_url": paper.pdf_url,
        })

    return results


In [4]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define the tools
google_search_tool = FunctionTool(
    google_search,
    description="Search Google for information, returns results with a snippet and body content"
)

arxiv_search_tool = FunctionTool(
    arxiv_search,
    description="Search Arxiv for papers related to a given topic, including abstracts"
)
# Define agents
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Google for information and provide enriched content",
    system_message="You are a helpful assistant skilled in performing web searches and summarizing results."
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="An agent that can search Arxiv for academic papers",
    system_message="You are a helpful assistant skilled in finding relevant academic papers."
)

report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a structured literature review based on the data collected",
    system_message="Your task is to synthesize data into a high-quality literature review with proper citations. End your response with 'TERMINATE'."
)


In [5]:

from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_core import CancellationToken


# Define termination condition
termination = TextMentionTermination("TERMINATE")

# Create the team
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent],
    termination_condition=termination
)


In [6]:
await Console(
    team.run_stream(
        task="Write a literature review on no code tools for the effect of AI on  Politic",
    )
)

---------- user ----------
Write a literature review on no code tools for building multi agent ai systems
---------- Google_Search_Agent ----------
[FunctionCall(id='call_30NUEJKvEY4u0V2Ut7rCg00X', arguments='{"query":"no code tools for building multi-agent AI systems","num_results":5}', name='google_search')]
---------- Google_Search_Agent ----------
[FunctionExecutionResult(content='[{\'title\': \'AutoGen Studio: A No-Code Developer Tool for Building and ...\', \'link\': \'https://arxiv.org/abs/2408.15247\', \'snippet\': \'Aug 9, 2024 ... Abstract:Multi-agent systems, where multiple agents (generative AI models + tools) collaborate, are emerging as an effective pattern for\\xa0...\', \'body\': \'[2408.15247] AutoGen Studio: A No-Code Developer Tool for Building and Debugging Multi-Agent Systems Skip to main content In just 3 minutes help us improve arXiv: Annual Global Survey We gratefully acknowledge support from the Simons Foundation, member institutions , and all contributors. Don

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a literature review on no code tools for building multi agent ai systems', type='TextMessage'), ToolCallRequestEvent(source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=125, completion_tokens=28), content=[FunctionCall(id='call_30NUEJKvEY4u0V2Ut7rCg00X', arguments='{"query":"no code tools for building multi-agent AI systems","num_results":5}', name='google_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Google_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content='[{\'title\': \'AutoGen Studio: A No-Code Developer Tool for Building and ...\', \'link\': \'https://arxiv.org/abs/2408.15247\', \'snippet\': \'Aug 9, 2024 ... Abstract:Multi-agent systems, where multiple agents (generative AI models + tools) collaborate, are emerging as an effective pattern for\\xa0...\', \'body\': \'[2408.15247] AutoGen Studio: A No-Code Developer Tool for Building 

In [ ]:
import asyncio
import time

# Example async function representing your agents' tasks
async def example_task(agent_name):
    print(f"{agent_name} starting task...")
    start_time = time.time()  # Capture the start time
    
    # Simulate task execution with sleep
    await asyncio.sleep(2)  # Simulate some task, like calling an agent
    
    end_time = time.time()  # Capture the end time
    task_duration = end_time - start_time  # Calculate the duration
    print(f"{agent_name} finished task in {task_duration:.2f} seconds.")
    return agent_name, task_duration  # Return the agent name and duration

# Main function for running the benchmark
async def run_benchmark():
    print("Running benchmark...")

    # List of agents you want to benchmark
    agents = ['PlanningAgent', 'Google_Search_Agent', 'Content_Creator', 'Trump_Agent', 'Social_Media_Agent']
    
    # Create a list of tasks
    tasks = [example_task(agent) for agent in agents]
    
    # Run the tasks concurrently and wait for them to complete
    results = await asyncio.gather(*tasks)
    
    # Print the results after all tasks are completed
    print("\nBenchmark Results:")
    for agent_name, task_duration in results:
        print(f"{agent_name}: {task_duration:.2f} seconds")

    print("Benchmark completed!")

# Check if running in an interactive environment (like Jupyter notebook)
if __name__ == "__main__":
    try:
        # Directly use await if in a running event loop (like Jupyter)
        await run_benchmark()
    except RuntimeError:
        # If there's no event loop, run with asyncio.run()
        asyncio.run(run_benchmark())
